In [1]:
import cv2
import pickle
import os
import cv2
import pickle
import numpy as np
import mediapipe as mp
import time
import os
import face_recognition
import cvzone
import sqlite3
from datetime import datetime

2023-12-05 22:00:38.997391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)

imgBackground=cv2.imread("resources/background.png")

# importing mode images into a list
folderModePath='resources/modes'
modePathList=os.listdir(folderModePath)
imgModeList=[]
for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath,path)))
    

#load the encoding file
file=open("EncodeFile.pkl","rb")
encodeListKnownWithIds=pickle.load(file)
file.close()
encodeListKnown,studentIds=encodeListKnownWithIds

#print(encodeListKnown,studentIds)
modeType=0
counter=0
id=-1

while True:
    success,img=cap.read()
    imgs=cv2.resize(img,(0,0),None,0.25,0.25)
    imgs=cv2.cvtColor(imgs,cv2.COLOR_BGR2RGB)
    faceCurrentFrame=face_recognition.face_locations(imgs)
    encodeCurrentFrame=face_recognition.face_encodings(imgs,faceCurrentFrame)
    
    imgBackground[162:162+480,55:55+640]=img
    imgBackground[44:44+633,808:808+414]=imgModeList[modeType]
    if faceCurrentFrame:
        for encodeFace,faceLoc in zip(encodeCurrentFrame,faceCurrentFrame):
            matches=face_recognition.compare_faces(encodeListKnown,encodeFace)
            faceDistance=face_recognition.face_distance(encodeListKnown,encodeFace)
            print("matches:",matches)
            print("faceDistance:",faceDistance)
            matchIndex=np.argmin(faceDistance)

            #print(matchIndex)
            if matches[matchIndex]:
                #print("known face detected")
                #print(studentIds[matchIndex])
                y1,x2,y2,x1=faceLoc
                y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
                bbox=55+x1,162+y1,x2-x1,y2-y1
                imgBackground=cvzone.cornerRect(imgBackground,bbox)

                id=studentIds[matchIndex]

                if counter==0:
                    counter=1
                    modeType=1

        if counter!=0:

            if counter==1:
                # get data from database
                connection=sqlite3.connect("Attendence.db")
                cursor=connection.cursor()
                cursor.execute('select * from attendenceDetails where ID=?',(id,))
                s=cursor.fetchall()
                print(id)
                print(s)
                name=s[0][1]
                major=s[0][2]
                start_year=s[0][3]
                total_attend=s[0][4]
                standing=s[0][5]
                year=s[0][6]
                last_attend=s[0][7]

                connection.commit()
                connection.close()

                # get image 
                path='images'
                modePathList1=os.listdir(path)
                for i in modePathList1:     
                    if(os.path.splitext(i)[0]==str(id)):
                        outputImg=cv2.imread(os.path.join(path,i))


                datetimeObject=datetime.strptime(last_attend,"%Y-%m-%d %H:%M:%S")
                secondsElapse=(datetime.now()-datetimeObject).total_seconds()
                print(secondsElapse)
                if secondsElapse>30:
                    #update attendence
                    connection=sqlite3.connect("Attendence.db")
                    cursor=connection.cursor()
                    cursor.execute('update attendenceDetails set total_attendence=?+1 where ID=?',(total_attend,id))

                    connection.commit()
                    connection.close()
                    #update last_attendence
                    connection=sqlite3.connect("Attendence.db")
                    cursor=connection.cursor()

                    curtime=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    cursor.execute('update attendenceDetails set last_attendence=? where ID=?',(curtime,id))

                    connection.commit()
                    connection.close()
                    modeType=1
                    counter=1
                else:
                    modeType=3
                    counter=0
                    imgBackground[44:44+633,808:808+414]=imgModeList[modeType]

            if modeType!=3:
                if 10<counter<20:
                    modeType=2

                imgBackground[44:44+633,808:808+414]=imgModeList[modeType]

                if counter<=10:
                    cv2.putText(imgBackground,str(total_attend),(861,125),cv2.FONT_HERSHEY_COMPLEX,0.65,(255,255,255),1)

                    (w,h),_=cv2.getTextSize(name,cv2.FONT_HERSHEY_COMPLEX,0.65,1)
                    offset=(414-w)//2
                    cv2.putText(imgBackground,str(name),(808+offset,445),cv2.FONT_HERSHEY_COMPLEX,0.65,(50,50,50),1)
                    cv2.putText(imgBackground,str(major),(1006,550),cv2.FONT_HERSHEY_COMPLEX,0.65,(255,255,255),1)
                    cv2.putText(imgBackground,str(id),(1006,493),cv2.FONT_HERSHEY_COMPLEX,0.65,(255,255,255),1)
                    cv2.putText(imgBackground,str(standing),(910,625),cv2.FONT_HERSHEY_COMPLEX,0.65,(100,100,100),1)
                    cv2.putText(imgBackground,str(year),(1025,625),cv2.FONT_HERSHEY_COMPLEX,0.65,(100,100,100),1)
                    cv2.putText(imgBackground,str(start_year),(1125,625),cv2.FONT_HERSHEY_COMPLEX,0.65,(100,100,100),1)

                    imgBackground[175:175+216,909:909+216]=outputImg

            counter+=1
            if counter>=20:
                counter=0
                modeType=0              
                imgBackground[44:44+633,808:808+414]=imgModeList[modeType]
        
    else:
        modeType=0
        counter=0
        
    cv2.imshow("Face Attendence",imgBackground)    
    if(cv2.waitKey(1)==ord("q")):
        break
        
cap.release()
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0xd395560) is not the object's thread (0xd19c7e0).
Cannot move to target thread (0xd395560)

QObject::moveToThread: Current thread (0xd395560) is not the object's thread (0xd19c7e0).
Cannot move to target thread (0xd395560)

QObject::moveToThread: Current thread (0xd395560) is not the object's thread (0xd19c7e0).
Cannot move to target thread (0xd395560)

QObject::moveToThread: Current thread (0xd395560) is not the object's thread (0xd19c7e0).
Cannot move to target thread (0xd395560)

QObject::moveToThread: Current thread (0xd395560) is not the object's thread (0xd19c7e0).
Cannot move to target thread (0xd395560)

QObject::moveToThread: Current thread (0xd395560) is not the object's thread (0xd19c7e0).
Cannot move to target thread (0xd395560)

QObject::moveToThread: Current thread (0xd395560) is not the object's thread (0xd19c7e0).
Cannot move to target thread (0xd395560)

QObject::moveToThread: Current thread (0xd395560) is not the object's thread

QObject::moveToThread: Current thread (0xd395560) is not the object's thread (0xd19c7e0).
Cannot move to target thread (0xd395560)

QObject::moveToThread: Current thread (0xd395560) is not the object's thread (0xd19c7e0).
Cannot move to target thread (0xd395560)



matches: [False, False, True, False, True, False]
faceDistance: [0.95244985 0.68278849 0.58775415 0.75689357 0.5769712  0.63580072]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 7, 'G', 4, '2023-12-03 16:41:22')]
192034.431116
matches: [False, False, True, False, True, False]
faceDistance: [0.97735639 0.66436633 0.53633272 0.81983803 0.53614359 0.64876754]
matches: [False, True, True, False, True, False]
faceDistance: [0.92791027 0.54293976 0.55320768 0.74478916 0.47555759 0.66804671]
matches: [False, True, False, False, True, False]
faceDistance: [0.91302776 0.56337615 0.65741172 0.70038515 0.5230923  0.69024188]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
1.733466
matches: [False, True, False, False, True, False]
faceDistance: [0.95426972 0.58389191 0.67101878 0.63347822 0.49134714 0.67202882]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
2.061062
matches: [False, True, True, False, True, False]

matches: [False, True, False, False, True, False]
faceDistance: [0.83774815 0.54467511 0.62655548 0.72606436 0.52258537 0.67903878]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
12.752427
matches: [False, False, False, False, True, False]
faceDistance: [0.88790077 0.62851079 0.68154547 0.69707132 0.55874435 0.64660306]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
13.077425
matches: [False, True, False, False, True, False]
faceDistance: [0.83138886 0.59437791 0.6529741  0.67295451 0.55203732 0.67721182]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
13.405311
matches: [False, False, False, False, True, False]
faceDistance: [0.87666216 0.6198048  0.65710084 0.64352308 0.53568045 0.67799487]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
13.727089
matches: [False, True, False, False, True, False]
faceDistance: [0.86397242 0.4949

matches: [False, True, True, False, True, False]
faceDistance: [0.93616803 0.56118168 0.52408002 0.74291831 0.44670976 0.66509602]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
26.776815
matches: [False, True, True, False, True, False]
faceDistance: [0.86264423 0.582606   0.52620718 0.69334538 0.46104974 0.65096097]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
27.100524
matches: [False, False, True, False, True, False]
faceDistance: [0.87364763 0.63672969 0.58394262 0.77025539 0.49300247 0.69878466]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
27.423263
matches: [False, True, True, False, True, False]
faceDistance: [0.84911673 0.56479783 0.5260392  0.72709227 0.4576432  0.6601119 ]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 8, 'G', 4, '2023-12-05 22:01:56')]
27.748619
matches: [False, True, True, False, True, False]
faceDistance: [0.88234371 0.58060376 0

...

matches: [False, True, False, False, True, False]
faceDistance: [0.91000881 0.5954544  0.61231496 0.70301714 0.53179965 0.64622108]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 18, 'G', 4, '2023-12-05 22:07:33')]
11.312724
matches: [False, False, True, False, True, False]
faceDistance: [0.94609859 0.61158204 0.57463568 0.73596004 0.50302006 0.63628624]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 18, 'G', 4, '2023-12-05 22:07:33')]
11.676363
matches: [False, False, False, False, True, False]
faceDistance: [0.92695466 0.61219044 0.60748285 0.70692885 0.55232661 0.62063347]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 18, 'G', 4, '2023-12-05 22:07:33')]
12.035981
matches: [False, True, False, False, True, False]
faceDistance: [0.85858059 0.54447275 0.6385258  0.66768472 0.51001271 0.63601152]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 18, 'G', 4, '2023-12-05 22:07:33')]
12.388919
matches: [False, False, True, False, True, False]
faceDistance: [0.88083424 0.7

matches: [False, False, True, False, True, True]
faceDistance: [0.89824483 0.61875074 0.59373326 0.71216041 0.48512114 0.58515839]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 18, 'G', 4, '2023-12-05 22:07:33')]
26.551564
matches: [False, False, True, False, True, False]
faceDistance: [0.87447304 0.61467259 0.57972011 0.72913354 0.54512073 0.66041022]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 18, 'G', 4, '2023-12-05 22:07:33')]
27.015655
matches: [False, False, True, False, True, False]
faceDistance: [0.90050654 0.65069409 0.57959657 0.71144744 0.55303683 0.61841146]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 18, 'G', 4, '2023-12-05 22:07:33')]
27.374134
matches: [False, True, False, False, True, False]
faceDistance: [0.9497448  0.59778898 0.61233328 0.75274311 0.48353616 0.61411632]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 18, 'G', 4, '2023-12-05 22:07:33')]
27.732295
matches: [False, False, True, False, True, False]
faceDistance: [0.96098631 0.698

matches: [False, False, True, False, True, False]
faceDistance: [0.92849149 0.60960085 0.58046906 0.72219256 0.49524752 0.62055205]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 19, 'G', 4, '2023-12-05 22:08:03')]
11.052116
matches: [False, False, False, False, True, False]
faceDistance: [0.87941562 0.67551929 0.6430582  0.76678274 0.5889288  0.70503121]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 19, 'G', 4, '2023-12-05 22:08:03')]
11.521537
matches: [False, False, False, False, True, False]
faceDistance: [0.95218079 0.60627502 0.61231728 0.75331777 0.53937345 0.6348206 ]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 19, 'G', 4, '2023-12-05 22:08:03')]
11.863379
matches: [False, False, True, False, True, False]
faceDistance: [0.9079885  0.6463297  0.59958179 0.74066174 0.566779   0.66961537]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 19, 'G', 4, '2023-12-05 22:08:03')]
12.207052
matches: [False, False, True, False, True, False]
faceDistance: [0.89054085 0.

matches: [False, True, False, False, True, False]
faceDistance: [0.9360185  0.56821765 0.60348021 0.70739359 0.46923706 0.64691092]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 19, 'G', 4, '2023-12-05 22:08:03')]
24.876842
matches: [False, True, True, False, True, False]
faceDistance: [0.91426926 0.54771421 0.58984217 0.68782302 0.453953   0.65262519]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 19, 'G', 4, '2023-12-05 22:08:03')]
25.255794
matches: [False, False, True, False, True, False]
faceDistance: [0.92116156 0.60934987 0.59526027 0.71105118 0.47559869 0.65563475]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 19, 'G', 4, '2023-12-05 22:08:03')]
25.648028
matches: [False, True, True, False, True, False]
faceDistance: [0.86922675 0.55752769 0.59054412 0.67456582 0.45692562 0.63481075]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 19, 'G', 4, '2023-12-05 22:08:03')]
26.039586
matches: [False, True, True, False, True, False]
faceDistance: [0.88207453 0.56228

matches: [False, False, True, False, True, False]
faceDistance: [0.93857282 0.61090187 0.54310757 0.75219969 0.51120718 0.61797561]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 20, 'G', 4, '2023-12-05 22:08:33')]
13.405466
matches: [False, False, True, False, True, False]
faceDistance: [0.94379942 0.61004026 0.56968643 0.74730153 0.49021485 0.62655961]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 20, 'G', 4, '2023-12-05 22:08:33')]
13.776431
matches: [False, True, True, False, True, False]
faceDistance: [0.93284773 0.55910846 0.57269429 0.74299809 0.47046803 0.62785621]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 20, 'G', 4, '2023-12-05 22:08:33')]
14.133857
matches: [False, False, True, False, True, False]
faceDistance: [0.93338794 0.61195696 0.56512843 0.76019154 0.4763219  0.63513316]
190703
[('190703', 'BHAVANA ANNAVARAM', 'AI', 2019, 20, 'G', 4, '2023-12-05 22:08:33')]
14.49365
matches: [False, True, True, False, True, False]
faceDistance: [0.9386264  0.58478